In [ ]:
import os
import re
import sys
import numpy as np
import pandas as pd
import cmath
import matplotlib.pyplot as plt
import seaborn as sns

powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2020 SP4\Python\3.8'
if powerfactory_path not in sys.path:
    sys.path.append(powerfactory_path)
import powerfactory as pf

try:
    from pfcommon import *
except:
    sys.path.append('..')
    from pfcommon import *

In [ ]:
def find_in_contents(container, name):
    for obj in container.GetContents():
        if obj.loc_name == name:
            return obj
    return None

class StochasticLoad (object):
    def __init__(self, load, grid, app, n_samples, P=None, Q=None, outdir='.'):
        self.load = load
        self.grid = grid
        self.app = app
        self.n_samples = n_samples
        if P is not None:
            self.Pm,self.Ps = P
        else:
            self.Pm,self.Ps = load.plini, load.plini/100*5
        if Q is not None:
            self.Qm,self.Qs = Q
        else:
            self.Qm,self.Qs = load.qlini, load.qlini/100*5
        self.meas_filepath = os.path.join(outdir, load.loc_name.replace(' ', '_') + '_PQ.dat')

        library = find_in_contents(self.app.GetActiveProject(), 'Library')
        if library is None:
            raise Exception('Cannot locate library')
        user_models = find_in_contents(library, 'User Defined Models')
        if user_models is None:
            raise Exception('Cannot locate user models')
        self.frame = find_in_contents(user_models, 'TimeVaryingLoadFrame')
        if self.frame is None:
            raise Exception('Cannot locate time-varying load frame')

    def _write_load_file(self, store_tPQ=False):
        tPQ = np.zeros((n_samples, 3))
        tPQ[:,0] = t
        tPQ[:,1] = self.Pm + self.Ps * np.random.normal(size=self.n_samples)
        tPQ[:,2] = self.Qm + self.Qs * np.random.normal(size=self.n_samples)
        with open(self.meas_filepath, 'w') as fid:
            fid.write('2\n\n')
            for row in tPQ:
                fid.write(f'{row[0]:.6f}\t{row[1]:.2f}\t{row[2]:.2f}\n\n')
        if store_tPQ:
            self.tPQ = tPQ

    def build(self):
        ld_name = self.load.loc_name.replace(' ', '_')
        self._write_load_file(store_tPQ=True)
        self.meas_file = self.grid.CreateObject('ElmFile', 'meas_' + ld_name)
        self.meas_file.f_name = self.meas_filepath
        self.comp_model = grid.CreateObject('ElmComp', 'stochastic_' + ld_name)
        self.comp_model.typ_id = self.frame
        self.comp_model.SetAttribute("pelm", [self.meas_file, self.load])
        
    def clean(self):
        self.meas_file.Delete()
        self.comp_model.Delete()

#### Get the PowerFactory application

In [ ]:
app = pf.GetApplication()
if app is None:
    raise Exception('Cannot get PowerFactory application')
print('Got PowerFactory application.')

#### Activate the project
We start with the simple 9-bus system, which contains only three loads.

In [ ]:
project_name = '\\Terna_Inerzia\\Nine-bus System'
err = app.ActivateProject(project_name)
if err:
    raise Exception(f'Cannot activate project {project_name}')
print(f'Activated project "{project_name}".')

#### Get the active project

In [ ]:
project = app.GetActiveProject()
if project is None:
    raise Exception('Cannot get active project')
print('Got active project.')

#### Get some info on the network

In [ ]:
generators = app.GetCalcRelevantObjects('*.ElmSym')
lines = app.GetCalcRelevantObjects('*.ElmLne')
buses = app.GetCalcRelevantObjects('*.ElmTerm')
loads = app.GetCalcRelevantObjects('*.ElmLod')
transformers = app.GetCalcRelevantObjects('*.ElmTr2')
n_generators, n_lines, n_buses = len(generators), len(lines), len(buses)
n_loads, n_transformers = len(loads), len(transformers)
print(f'There are {n_generators} generators.')
print(f'There are {n_lines} lines.')
print(f'There are {n_buses} buses.')
print(f'There are {n_loads} loads.')
print(f'There are {n_transformers} transformers.')

#### Run a power flow analysis
This is just to make sure that everything works fine.

In [ ]:
study_cases_proj_folder = app.GetProjectFolder('study')
if study_cases_proj_folder is None:
    raise Exception('Cannot get the study cases project folder')
print('Got study cases project folder.')
PF_study_case_name = '01- Load Flow.IntCase'
study_cases = study_cases_proj_folder.GetContents(PF_study_case_name)
if len(study_cases) == 0:
    raise Exception(f'Cannot get study case "{PF-study_case_name}".')
PF_study_case = study_cases[0]
print(f'Got study case "{PF_study_case_name}".')
err = PF_study_case.Activate()
if err:
    print(f'Study case "{PF_study_case_name}" was already activated.')
else:
    print(f'Activated study case "{PF_study_case_name}".')

In [ ]:
lf_res = run_power_flow(app, study_cases_proj_folder, PF_study_case_name, generators,
                       loads, buses, lines, transformers)
print_power_flow(lf_res)

#### Make all the loads in the network stochastic
First of all, we activate the study case called `09- Stochastic Loads`.

In [ ]:
study_case_name = '09- Stochastic Loads'
study_case = study_cases_proj_folder.GetContents(study_case_name)[0]
err = study_case.Activate()
if err:
    raise Exception(f'Cannot activate study case {study_case_name}')

In [ ]:
srate = 1000.
dt = 1/srate
tend = 100
t = np.r_[0 : tend : dt]
n_samples = t.size

In [ ]:
stochastic = True
if stochastic:
    grids = app.GetCalcRelevantObjects('*.ElmNet')
    grid = grids[0]
    loads = app.GetCalcRelevantObjects('*.ElmLod')
    stoch_loads = []
    for load in loads:
        stoch_load = StochasticLoad(load, grid, app, n_samples,
                                    outdir='C:\\Users\\Terna_Inerzia\\Desktop\\ai-pf\\PF')
        stoch_load.build()
        stoch_loads.append(stoch_load)

#### Compute the initial condition of the simulation

In [ ]:
inc = app.GetFromStudyCase('ComInc')
inc.iopt_sim = 'rms'
inc.iopt_coiref = 2
inc.tstart = 0
inc.dtgrd = dt
err = inc.Execute()
if err:
    raise Exception('Cannot compute initial condition')

#### Tell PowerFactory which variables should be saved to its internal file

In [ ]:
# speed, mechanical torque, electrical torque, terminal voltage, electrical power
var_names = 's:xspeed', #'s:xme', 's:xmt', 's:ut', 's:pgt'
res = app.GetFromStudyCase('*.ElmRes')
for gen in generators:
    for var_name in var_names:
        res.AddVariable(gen, var_name)

#### Run the transient simulation

In [ ]:
sim = app.GetFromStudyCase('ComSim')
sim.tstop = tend
err = sim.Execute()
if err:
    raise Exception('Cannot run transient simulation')

#### Get the data

In [ ]:
res.Load()
time = get_simulation_time(res)
data = {}
for var_name in var_names:
    data[var_name] = get_simulation_variables(res, var_name, elements=generators)

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6,4))
cmap = plt.get_cmap('viridis', n_generators)
for i in range(n_generators):
    ax.plot(time, data['s:xspeed'][:,i], color=cmap(i), lw=1)
sns.despine()
ax.set_xlabel('Time [s]')
ax.set_ylabel('ω [p.u.]')
ax.set_ylim([0.9964,0.9972])
fig.tight_layout()

#### Remove the composite models that were added previously

In [ ]:
do_remove = True
if stochastic and do_remove:
    for load in stoch_loads:
        load.clean()